# Lithology attributes extraction


Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the lithological attributes from the GLiM shapefile to the catchment boundaries.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made avaialable in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* rasterio
* time
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/lithology/GLiM.shp. Available at: http://dx.doi.org/10.1594/PANGAEA.788537 (Last access 23 November 2023)
* data/lithology/average_soil_and_sedimentary-deposit_thickness.tif. Available at: https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1304 (Last access 23 November 2023)
* data/shapefiles/estreams_boundaries.shp

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Hartmann, J., and Moosdorf, N. (2012), The new global lithological map database GLiM: A representation of rock properties at the Earth surface, Geochem. Geophys. Geosyst., 13, Q12004, https://doi.org/10.1029/2012GC004370

* Pelletier, J.D., P.D. Broxton, P. Hazenberg, X. Zeng, P.A. Troch, G. Niu, Z.C. Williams, M.A. Brunke, and D. Gochis. 2016. Global 1-km Gridded Thickness of Soil, Regolith, and Sedimentary Deposit Layers. ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/1304


## Licenses

* GLiM: Creative Commons Attribution 3.0 Unported (CC-BY-3.0). https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1304 (Last access 27 November 2023)
* Depth to bedrock: Open-access. https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1304 (Last access 27 November 2023)


## Observations

#### GLiM lithological classes

1. nd: No Data
2. su: Unconsolidated Sediments
3. ss: Siliciclastic Sedimentary 
4. sm: Mixed Sedimentary Rocks
5. sc: Carbonate Sedimentary Rocks
6. py: Pyroclastics
7. ev: Evaporites
8. mt: Metamorphic Rocks
9. pa: Acid Plutonic Rocks
10. pi: Intermediate Plutonic Rocks
11. pb: Basic Plutonic Rocks
12. va: Acid Volcanic Rocks
13. vi: Intermediate Vulcanic Rocks
14. vb: Basic Volcanic Rocks
15. ig: Ice and Glaciers
16. wb: Water bodies

# Import modules

In [ ]:
import geopandas as gpd
import pandas as pd
import tqdm as tqdm
import os
import numpy as np
import rasterio
import time
from rasterio.features import geometry_mask

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.


In [ ]:
# Non-editable variables
PATH = "../../.."
PATH_OUTPUT = "results/staticattributes/"
# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [ ]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

In [ ]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

## Depth to bedrock raster

In [ ]:
# Specify the path to the raster file you want to open
raster_depthtobedrock = "data/lithology/average_soil_and_sedimentary-deposit_thickness.tif"

## GLiM shapefile

GLiM original layer. Note that you can read an already clipped version and speed-up the processing.


In [ ]:
GLiM = gpd.read_file('data/lithology/GLiM.shp')
GLiM

To optimize the process it is important to dissolve the polygon geometries before intersecting the areas. Here we dissove it by the atribute field corresponding to the unique-id for each lithological class. 

In [ ]:
attribute_field = 'xx'
GLiM_dissolved = GLiM.dissolve(by=attribute_field)

# Now we create a new feature with the lithology class:
GLiM_dissolved["class"] = GLiM_dissolved.index
GLiM_dissolved

## Reproject to projected coordinates system

In [ ]:
# Here you can check the crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries.crs)
print("CRS of GLiM:", GLiM_dissolved.crs)

In [ ]:
# Define the target CRS to ETRS89 LAEA (3035)
target_crs = 'EPSG:3035'  

# Reproject the GeoDataFrame to the target CRS
catchment_boundaries_reprojected = catchment_boundaries.to_crs(target_crs)
GLiM_reprojected = GLiM_dissolved.to_crs(target_crs)

In [ ]:
# Here you can check the crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries_reprojected.crs)
print("CRS of GLiM:", GLiM_reprojected.crs)

# Bedrock depth extraction

In [ ]:
# For the bedrock we use the crs in 4326:
subset_catchment=catchment_boundaries.copy()

In [ ]:
# Create lists to store the results
avg_values = []
with rasterio.open(raster_depthtobedrock) as src:
    for idx, geom in tqdm.tqdm(subset_catchment.iterrows()):
        # Create a mask for the geometry
        mask = geometry_mask([geom['geometry']], out_shape=src.shape, transform=src.transform, invert=True)

        # Read the values within the geometry from the raster
        values = src.read(1, masked=True)
        values = values[mask]

        # Calculate statistics only if there are valid values in the 'values' array
        if len(values) > 0:
            avg_value = np.mean(values)

        else:
            # Handle the case when there are no valid values (e.g., by setting them to NaN or a specific value)
            avg_value = np.nan

        # Store the results in the lists
        avg_values.append(avg_value)

# Create a DataFrame to store the results for this file
data = {
    'basin_id': subset_catchment['basin_id'],
    'bedrk_dep': avg_values,
}
bedrk_dep_df = pd.DataFrame(data)
bedrk_dep_df.set_index("basin_id", inplace=True)


In [ ]:
bedrk_dep_df

# Intersection areas

In [ ]:
subset_catchment=catchment_boundaries_reprojected.copy()
subset_catchment

In [ ]:
# Record the start time
start_time = time.time()

lithology_overlap = gpd.overlay(df1=subset_catchment, df2=GLiM_reprojected, how='intersection')

# Record the end time
end_time = time.time()

# Print the elapsed time in seconds when done:
print("Elapsed time: {:.1f} seconds".format(end_time - start_time))

In [ ]:
# Calculate the areas of the overlapping polygons and add them as a new column
lithology_overlap['area_sqm'] = lithology_overlap['geometry'].area/1000000
lithology_overlap

# Pivot table

In [ ]:
# Finally we can creatre a pivot-table with the percentage of each lithological class per catchment:

lithology_areas = pd.pivot_table(
    lithology_overlap,
    values='area_sqm',     # Replace with the actual column name for the area
    index='basin_id',      # Rows are based on 'basin_id'
    columns='class',       # Columns are based on 'class' (the class)
    aggfunc='sum',         # Sum the areas for each combination
    fill_value=0           # Replace NaN with 0
)

# Here we can sum to compute the total area of each catchment: 
lithology_areas.loc[:, "totalarea"] = lithology_areas.sum(axis = 1)
lithology_areas

## Catchment covered by shapefile
* Here we compute the total catchment area covered by the lithology shapefile:


In [ ]:
catchment_boundaries_reprojected.set_index('basin_id', inplace = True)

In [ ]:
lithology_areas['area_calc'] = catchment_boundaries_reprojected.area / 1000000
lithology_areas

In [ ]:
lithology_areas['tot_area'] = lithology_areas.totalarea / lithology_areas.area_calc
lithology_areas

# Data organization

In [ ]:
# Here we compute the lithology percentages from each class:
lithology_df = (lithology_areas.iloc[:, :16].div(lithology_areas['totalarea'], axis=0))*100
lithology_df = lithology_df.iloc[:, 0:-1]
lithology_df

In [ ]:
# Create a new column with the name of the column with the majority class
lithology_df['lit_dom'] = lithology_df.apply(lambda row: row.idxmax(), axis=1)

# Add "th_new_" as a prefix to all column names
lithology_df = lithology_df.add_prefix('lit_fra_')
lithology_df = lithology_df.rename(columns={'lit_fra_lit_dom': 'lit_dom'})

# Catchment ara covered by the lithology shapef
lithology_df['tot_area'] = (lithology_areas.tot_area)*100

# Concatenate the bedrock depth:
lithology_df["bedrk_dep"] = bedrk_dep_df

In [ ]:
# Here we sort the index:
lithology_df = lithology_df.sort_index(axis=0)
lithology_df

In [ ]:
# Round the data to 3 decimals:
lithology_df.iloc[:, 0:-3] = lithology_df.iloc[:, 0:-3].round(3)
lithology_df.iloc[:, -2:] = lithology_df.iloc[:, -2:].round(3)

lithology_df

# Data export

In [ ]:
# Export the final dataset:
lithology_df.to_csv(PATH_OUTPUT+"estreams_geology_attributes.csv")

# End